In [ ]:
#Download and unzip ID mapping file : 

wget -b https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping_selected.tab.gz

gunzip -c idmapping_selected.tab.gz > idmapping_selected.tab &


In [ ]:
#Download Uniprot TreEbml and Swissprot 

wget -b https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_trembl.tab.gz

gunzip -c uniport_trembl.tab.gz > uniport_trembl.tab &


wget -b https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.tab.gz

gunzip -c uniport_swissprot.tab.gz > uniport_swissprot.tab &


In [ ]:
#Extracting Accession ID, Uniref50 ID, Uniref90 ID, Uniref100 ID and EmblCdsId from idmapping file

import sys
import csv

csv.field_size_limit(sys.maxsize)

def extract_id(row, keyword):
    for element in row:
        if keyword in element:
            return element
    return None

def extract_ids_from_file(input_file, output_file):
    with open(input_file, 'r') as file:
        with open(output_file, 'w', newline='') as csvfile:
            csv_reader = csv.reader(file, delimiter='\t')
            csv_writer = csv.writer(csvfile)

            # Writing header
            csv_writer.writerow(['AC', 'UniRef100', 'UniRef90', 'UniRef50', 'EmblCdsId'])

            # Process each row
            for row in csv_reader:
                # Extracting the IDs based on keywords
                uniref100 = extract_id(row, 'UniRef100')
                uniref90 = extract_id(row, 'UniRef90')
                uniref50 = extract_id(row, 'UniRef50')
                
                emblcds = row[-1]
                ac = row[0]

                # Writing data
                csv_writer.writerow([ac, uniref100, uniref90, uniref50, emblcds])

    print(f"Extracted IDs written to {output_file}")

# Usage
input_file = 'idmapping_selected.tab'
output_file = 'idmapping.csv'
extract_ids_from_file(input_file, output_file)


In [ ]:
#Extract EC number(EC) and Accession ID(AC) for Bacteria & Archaea (OC)

import csv
import re

def extract_ac_ec_oc(input_file_path, output_file_path):
    
    # Open the input TSV file for reading
    with open(input_file_path, 'r', newline='') as file:
        # Open the output CSV file for writing
        with open(output_file_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['AC', 'EC', 'OC'])  # Write the header row to the CSV file

            for row in csv.reader(file, delimiter='\t'):
                superkingdom = ''  # Default value if no match is found in the line

                # Extract superkingdom from the taxonomic lineage
                superkingdom_match = re.search(r'(Bacteria|Archaea)', row[2])
                if superkingdom_match:
                    superkingdom = superkingdom_match.group(1)
                    entry = row[0]
                    ec_number = row[1]
                    writer.writerow([entry, ec_number, superkingdom])

# Extract Values from uniprot trembl 
extract_ac_ec_oc('uniport_trembl.tab', 'extracted_trembl.csv')
# Extract Values from uniprot swissprot 
extract_ac_ec_oc('uniport_swissprot.tab', 'extracted_swissprot.csv')


In [ ]:
# Drop records with blank EC values

import pandas as pd

def drop_blank_entries(input_csv, output_csv):
    try:
        # Read the CSV file
        df = pd.read_csv(input_csv)

        # Drop rows with any empty cells
        df.dropna(inplace=True)

        # Save the cleaned DataFrame to a new CSV file
        df.to_csv(output_csv, index=False)

        print("Blank entries dropped and saved successfully.")

    except Exception as e:
        print("An error occurred:", e)

# Usage for treml.csv
input_csv_path = "extracted_trembl.csv"
output_csv_path = "trembl.csv"
drop_blank_entries(input_csv_path, output_csv_path)

# Usage for swissprot.csv
input_csv_path = "extracted_swissprot.csv"
output_csv_path = "swissprot.csv"
drop_blank_entries(input_csv_path, output_csv_path)


In [ ]:
#Removing records where EC Number contains - (example : 1.2.1.-,1.-.-.-)

import csv

def remove_ec_numbers_with_dash(input_file, output_file):
    # Define a function to check if EC number contains any '-'
    def has_dash(ec_number):
        return '-' in ec_number

    # Open input and output files
    with open(input_file, "r") as infile, open(output_file, "w", newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        # Write the header to the output file
        header = next(reader)
        writer.writerow(header)

        # Process each line in the input file
        for row in reader:
            # Check if EC number has '-'
            if not has_dash(row[1]):  # Assuming EC number is in the second column (index 1)
                # Write the line to the output file
                writer.writerow(row)

# Usage
remove_ec_numbers_with_dash("trembl.csv", "cleaned_trembl.csv")
remove_ec_numbers_with_dash("swissprot.csv", "cleaned_swissprot.csv")


In [ ]:
# Merging idmapping file and Uniprot files
import pandas as pd

def merge_csv_files(file1, file2, output_file):

    # Read the CSV files
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Merge the DataFrames on the 'AC' column
    merged_df = pd.merge(df1, df2, on='AC')

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_file, index=False)

    print("Files have been successfully merged and saved to", output_file)

# Usage
file1 = './cleaned_trembl.csv'
file2 = './idmapping.csv'
output_file = './mapped_trembl.csv'
merge_csv_files(file1, file2, output_file)

file1 = './cleaned_swissprot.csv'
file2 = './idmapping.csv'
output_file = './mapped_swissprot.csv'
merge_csv_files(file1, file2, output_file)


In [ ]:
# For the records which contain more than one embl cds id in one field, split that entry into multiple rows, each row should contain one embl cds id
import pandas as pd

def split_embl_cds_id(file):

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Create a new DataFrame to store the split EmblCdsId values
    new_rows = []

    # Iterate through each row in the original DataFrame
    for index, row in df.iterrows():
        embl_ids = str(row['EmblCdsId']).split(';')  # Convert to string and then split by semicolon
        for embl_id in embl_ids:
            # Check if the value contains a hyphen
            if '-' not in embl_id:
                new_row = row.copy()
                new_row['EmblCdsId'] = embl_id
                new_rows.append(new_row)

    # Create a new DataFrame from the list of new rows
    new_df = pd.DataFrame(new_rows)

    # Update the original file with the new data
    new_df.to_csv(file_path, index=False)

# Usage:
file = './mapped_trembl.csv'
split_embl_cds_id(file)

file = './mapped_swissprot.csv'
split_embl_cds_id(file)

In [ ]:
# For the records which contain more than one EC number in one field, split that entry into multiple rows, each row should contain one EC Number
import pandas as pd

def split_ec_number(file):

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Create a new DataFrame to store the split EmblCdsId values
    new_rows = []

    # Iterate through each row in the original DataFrame
    for index, row in df.iterrows():
        ec_numbers = str(row['EC']).split(';')  # Convert to string and then split by semicolon
        for ec_number in ec_numbers:
            new_row = row.copy()
            new_row['EC'] = ec_number
            new_rows.append(new_row)

    # Create a new DataFrame from the list of new rows
    new_df = pd.DataFrame(new_rows)

    # Update the original file with the new data
    new_df.to_csv(file_path, index=False)

# Usage:
file = './mapped_trembl.csv'
split_ec_number(file)

file = './mapped_swissprot.csv'
split_ec_number(file)

In [ ]:
#Download sequences for each embl cds id

import pandas as pd
import requests
import certifi
import contextlib
from tqdm import tqdm
import numpy as np
from multiprocessing import Pool

def download_batch(ids):
    url_root = 'https://www.ebi.ac.uk/ena/browser/api/fasta/'
    url = url_root + ','.join(ids)
    try:
        response = requests.get(url, verify=certifi.where()).text
        return response
    except requests.exceptions.RequestException as e:
        print(f"Error making request for {','.join(ids)}: {e}")
        return ""
    except IOError as e:
        print(f"Error writing to file: {e}")
        return ""

def get_seqs(ids, output_file):
    batch_size = 500  # Adjust batch size as needed
    starts = list(np.arange(0, len(ids), batch_size))
    stops = list(np.arange(batch_size, len(ids), batch_size)) + [len(ids)]

    with open(output_file, 'a') as f:
        with contextlib.closing(f):
            with Pool(processes=4) as pool:  # Adjust the number of processes as needed
                results = list(tqdm(pool.imap(download_batch, [ids[start:stop] for start, stop in zip(starts, stops)]), total=len(starts)))

            for result in results:
                f.write(result)

# Usage
cds_id = pd.read_csv('mapped_trembl.csv')
get_seqs(cds_id['EmblCdsId'].str.replace(' ', ''), output_file= 'trembl_sequences.fasta')

cds_id = pd.read_csv('mapped_swissprot.csv')
get_seqs(cds_id['EmblCdsId'].str.replace(' ', ''), output_file= 'swissprot_sequences.fasta')



In [ ]:
""" Extract embl cds id and sequences from the downloaded fasta files. Create a final file which contains the colummns:
AC, EC, OC, UniRef50, UniRef90, UniRef100, EmblCdsId, Seq """
import pandas as pd
from Bio import SeqIO
import re

def extract_embl_cds_id(header):
    match = re.search(r'\|([^|]+)\|([^\s]+)', header)
    return match.group(2) if match else None

def retrieve_embl_cds_id_and_sequence(csv_file, fasta_file, output_file):
    # Read the CSV file
    df_csv = pd.read_csv(csv_file)
    
    # Create a dictionary to store EmblCdsId as keys and sequences as values
    sequence_dict = {}

    # Read the FASTA file and populate the dictionary
    for record in SeqIO.parse(fasta_file, 'fasta'):
        header = record.description
        embl_cds_id = extract_embl_cds_id(header)
        if embl_cds_id:
            sequence_dict[embl_cds_id] = str(record.seq)

    # Check if any EmblCdsId from the CSV file matches those in the FASTA dictionary
    matching_embl_cds_ids = set(df_csv['EmblCdsId']).intersection(sequence_dict.keys())

    if not matching_embl_cds_ids:
        print("No matching EmblCdsIds found.")
    else:
        # Add the 'Sequence' column based on the dictionary
        df_csv['Sequence'] = df_csv['EmblCdsId'].map(sequence_dict)

        # Filter the DataFrame based on matching EmblCdsIds
        df_merged = df_csv[df_csv['EmblCdsId'].isin(matching_embl_cds_ids)]

        # Rearrange columns with 'EmblCdsId' as the last column
        df_merged = df_merged[['AC','EC','OC','UniRef100','UniRef90','UniRef50','EmblCdsId','Sequence']]

        # Save the merged data to a new CSV file
        df_merged.to_csv(output_file, index=False)

# Usage
retrieve_embl_cds_id_and_sequence('mapped_trembl.csv', 'trembl_sequences.fasta', 'tremblwithsequences.csv')

retrieve_embl_cds_id_and_sequence('mapped_swissprot.csv', 'swissprot_sequences.fasta', 'swissprotwithsequences.csv')


In [ ]:
# PERFORMING SOME ANALYSIS BEFORE SPLITTING THE DATASETS AND CREATING BENCHMARKS

In [ ]:
# Calculating number of records for each EC number
import pandas as pd

def calculate_ec_counts(input_file, output_file, chunk_size):
    # Initialize a counter dictionary to store counts of EC values
    ec_counts = {}

    # Iterate over chunks of the file
    for chunk in pd.read_csv(input_file, chunk_size):
        # Count the occurrences of each EC value in the current chunk
        chunk_ec_counts = chunk['EC'].value_counts().to_dict()
        
        # Update the overall count dictionary with counts from the current chunk
        for ec, count in chunk_ec_counts.items():
            ec_counts[ec] = ec_counts.get(ec, 0) + count

    # Convert the dictionary to a DataFrame
    ec_counts_df = pd.DataFrame(list(ec_counts.items()), columns=['EC', 'Count'])

    # Save the DataFrame to a CSV file
    ec_counts_df.to_csv(output_csv, index=False)

# Usage:
calculate_ec_counts('tremblwithsequences.csv', 'trembl_ec_count.csv', 100000)
calculate_ec_counts('swissprotwithsequences.csv', 'swissprot_ec_count.csv', 10000)


In [ ]:
#Uniprot Trembl

import pandas as pd

trembl_df = pd.read_csv('tremblwithsequences.csv')
swissprot_df = pd.read_csv('swissprotwithsequences.csv')

ec_trembl = set(trembl_df['EC'])
ec_swissprot = set(swissprot_df['EC'])

# Find the differences
ec_numbers_only_in_trembl = ec_trembl - ec_swissprot
print(len(ec_numbers_only_in_trembl))

ec_numbers_only_in_swissprot = ec_swissprot - ec_trembl
print(len(ec_numbers_only_in_swissprot))


# Print the EC numbers with counts from file 1
print("EC numbers only in file 1 with counts:")
for ec_number in ec_numbers_only_in_trembl:
    count = ec_trembl[ec_trembl['EC'] == ec_number]['Count'].iloc[0]  # Assuming 'count' is the column name in file1.csv
    print(f"EC number: {ec_number}, Count: {count}")

In [ ]:
print("EC numbers only in file 2 with counts:")
for ec_number in ec_numbers_only_in_swissprot:
    count = ec_swissprot[ec_swissprot['EC'] == ec_number]['Count'].iloc[0]  # Assuming 'count' is the column name in file1.csv
    print(f"EC number: {ec_number}, Count: {count}")

In [ ]:
#Calculate median for trembl
pd.set_option('display.max_rows', 5000)

sorted_trembl = trembl_df.sort_values('Count')

display(sorted_trembl.style)

In [ ]:
sorted_trembl['Count'].median()

In [ ]:
# Finding median for Swissprot after removing ec number only present in Swiss

common_trembl_swiss = ec_swissprot - ec_numbers_only_in_swissprot

print(len(common_trembl_swiss))

# Print the EC numbers with counts from file 2
print("EC numbers common in swiss and uni with counts:")
for ec_number in common_trembl_swiss:
    count = df2[df2['EC'] == ec_number]['Count'].iloc[0]  # Assuming 'count' is the column name in file1.csv
    print(f"EC number: {ec_number}, Count: {count}")
    sum += count
    
print(sum)

# sorted_common_uni_swiss = common_uni_swiss.sort_values('Count')



In [ ]:
#swissprot ALL 

pd.set_option('display.max_rows', 5000)

sorted_swissprot = swissprot_df.sort_values('Count')

display(sorted_swissprot.style)

In [ ]:
filtered_df = df2[df2['EC'].isin(common_uni_swiss)]

len(filtered_df)
sorted = filtered_df.sort_values('Count')
sorted['Count'].median()

In [ ]:
#seperating records which were only in Swiss not Uni

import pandas as pd

# Load the data
file1_path = '.csv'
file1_df = pd.read_csv(file1_path)

# Assuming you have a set 'ec_numbers_only_in_file2' containing EC numbers
# Extract EC numbers from set 'ec_numbers_only_in_file2'
ec_numbers = ec_numbers_only_in_file2

# Filter file1_df to get rows where 'EC' column contains EC numbers from 'ec_numbers'
filtered_df = file1_df[file1_df['EC'].isin(ec_numbers)]

# Get the remaining data
remaining_df = file1_df[~file1_df['EC'].isin(ec_numbers)]

# Save the filtered data to a new file
filtered_file_path = 'only_in_swiss.csv'
filtered_df.to_csv(filtered_file_path, index=False)

# Save the remaining data to another file
remaining_file_path = 'common_in_swiss_trembl.csv'
remaining_df.to_csv(remaining_file_path, index=False)

print("Filtered data and remaining data have been saved successfully.")


In [ ]:
# Final Datasets: tremblwithsequences.csv 
# tremblwithsequences.csv was divided in two datasets as shown above : only_in_swiss.csv(This becomes part of test 2) and 
# common_in_swiss_trembl.csv(This is used as Swissprot only dataset)

# For balancing the datasets we used median = 10 for Swissprot only dataset and median = 250 for Swissprot + Trembl datset